In [ ]:
import torch
torch.cuda.is_available()


True

In [ ]:
!pip install torch transformers sentencepiece sacrebleu datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.8/100.8 kB 8.0 MB/s eta 0:00:00


# New section

In [ ]:
import torch
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cuda')

In [ ]:
model_name = "Helsinki-NLP/opus-mt-en-hi"

tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/812k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/306M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/258 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/306M [00:00<?, ?B/s]

The tied weights mapping and config for this model specifies to tie model.shared.weight to model.decoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie model.shared.weight to model.encoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
model.to(device)


MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(61950, 512, padding_idx=61949)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(61950, 512, padding_idx=61949)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLU()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05

In [ ]:
next(model.parameters()).device


device(type='cuda', index=0)

In [ ]:
df = pd.read_csv("/content/Hindi_English_Truncated_Corpus.csv")
df = df.dropna()
df.head()


,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what ...,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर..."
1,ted,"I'd like to tell you about one such child,",मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...
2,indic2012,This percentage is even greater than the perce...,यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।
3,ted,what we really mean is that they're bad at not...,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
4,indic2012,.The ending portion of these Vedas is called U...,इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।


In [ ]:
def translate_en_to_hi(text):
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=64
    ).to(device)

    outputs = model.generate(
        **inputs,
        max_length=64,
        num_beams=5
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [ ]:
print(translate_en_to_hi("Admission notice released"))
print(translate_en_to_hi("Last date for fee payment"))
print(translate_en_to_hi("My name is abhimanyu"))
print(translate_en_to_hi("age is 33"))
print(translate_en_to_hi("A computer is an electronic device that accepts data as input, processes it according to a set of instructions, and produces meaningful output. It can perform arithmetic and logical operations at very high speed and store large amounts of information. Computers are widely used in education, business, healthcare, communication, and scientific research due to their accuracy, efficiency, and versatility"))
print(translate_en_to_hi("A phone is an electronic device used for communication. It allows people to make calls, send messages, and—if it’s a smartphone—access the internet, apps, camera, music, and videos. Phones help people stay connected anytime and anywhere."))
print(translate_en_to_hi("You mother fucker"))
print(translate_en_to_hi("I don't have girlfriend"))


एडविड सूचना रिलीज़ की गई
भुगतान के लिए अंतिम तिथि
मेरा नाम एक अरब्गा है
उम्र 33 है
कंप्यूटर एक इलेक्ट्रॉनिक उपकरण है जो डेटा को इनपुट के रूप में स्वीकार करता है, निर्देशों के अनुसार कार्य करता है, और अर्थपूर्ण आउटपुट उत्पन्‍न करता है ।
फोन एक इलेक्ट्रॉनिक उपकरण है जो संचार के लिए इस्तेमाल किया जाता है. यह लोगों को कॉल करने, संदेश भेजने, और - यदि यह एक स्मार्टफोन है - इंटरनेट, साधन, कैमरा, संगीत, और वीडियो को एक्सेस कर सकता है. फोन लोगों को किसी भी समय और कहीं भी कहीं से जुड़े रहने में
तुम माँ कमीने
मैं प्रेमिका नहीं है
